In [26]:
import pandas as pd
pd.set_option('display.max_rows', 10)

month="Aug"
month_num="08"
year="2024"

description_col = 'Security Description'
security_col = 'Security No.'
symbol_col = 'Symbol'
exchange_col = 'Exchange'
currency_col = 'Currency'
holding_col = f'Holding {month}{year}'
value_col = 'Market Value'
percent_col = '% of Fund'
jpm_sp_col = 'SP JPM GBP'
last_sp_col = f'SP {month}{year} lc'
today_sp_col = 'SP Today lc'

In [27]:
jema_symbols = pd.read_csv('./jema_symbols_exchanges.csv').dropna(subset=[symbol_col])
# jema_symbols[holding_col] = jema_symbols[holding_col].str.replace(',', '').astype(float)
jema_symbols.reset_index(inplace=True, drop=True)
jema_symbols

,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate
0,ABSA GROUP LTD,BFX05H3,ABG,JSE,ZAC,1
1,ABU DHABI COMMERCIAL BANK PJSC COMMON STOCK AED 1,6545464,ADCB,ADX,AED,1
2,ABU DHABI ISLAMIC BANK PJSC COMMON STOCK AED 1,6001728,ADIB,ADX,AED,1
3,ADES HOLDING CO COMMON STOCK SAR 1,BR56KM3,2382,TADAWUL,SAR,1
4,ADNOC DRILLING CO PJSC COMMON STOCK AED,BN12D39,ADNOCDRILL,ADX,AED,1
...,...,...,...,...,...,...
97,UNITED INTERNATIONAL TRANSPORTATION CO COMMON ...,B2493D0,4260,TADAWUL,SAR,1
98,VTB BANK PJSC COMMON STOCK RUB 0.01,0H5837S,VTBR,ALOR,RUB,1
99,X 5 RETAIL GROUP NV-REGS GDR,0H6365S,FIVE,ALOR,RUB,1
100,YANDEX NV COMMON STOCK USD 0.01,0H6379S,YDEX,ALOR,RUB,1


In [28]:
month_sheet = f"August {year}"
jema_data = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name=month_sheet, skiprows=9, usecols=range(5))
jema_data = jema_data.dropna()
jema_data = jema_data.drop(columns=[description_col, percent_col])
jema_data = jema_data.rename(columns={'Holding': holding_col})
jema_data[value_col] = jema_data[value_col].astype(float)
jema_data

,Holding Aug2024,Market Value,Security No.
1,164763,607097.53,6606996
2,32585,582080.93,B12LZH9
3,514330.73,514330.73,5819115
4,47354,487024.88,B030GJ7
5,142756,469868.43,6148197
...,...,...,...
112,12313057,14099.33,0H5808S
113,577525,4130.68,0H6372S
114,75226,1827.64,0H6380S
115,275182,1112.67,0H6365S


In [29]:
symbols_with_curr_vals = pd.merge(jema_symbols, jema_data, on=security_col, how='left')

holdings_to_sum_vals = {
    "LKOH": ("2H6442S", "0H5797S"),
    "GAZP": ("0H5796S", "0H6364S"),
    "SBER": ("0H5856S", "0H5855S"),
    "NVTK": ("2H6464S", "0H5828S"),
    "ROSN": ("0H5827S", "2H7674S"),
    "TATN": ("0H5822S", "0H5816S"),
}

for symbol, (sec_no_1, sec_no_2) in holdings_to_sum_vals.items():
    val_sum = jema_data.loc[jema_data[security_col] == sec_no_1][value_col].item() + jema_data.loc[jema_data[security_col] == sec_no_2][value_col].item()
    symbols_with_curr_vals.loc[symbols_with_curr_vals[symbol_col] == symbol, value_col] = val_sum
    display(symbols_with_curr_vals.loc[symbols_with_curr_vals[symbol_col] == symbol])


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Aug2024,Market Value
47,LUKOIL PJSC TOTAL,NaN,LKOH,ALOR,RUB,1,NaN,281208.46


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Aug2024,Market Value
35,GAZPROM PJSC TOTAL,NaN,GAZP,ALOR,RUB,1,NaN,217900.81


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Aug2024,Market Value
83,SBERBANK OF RUSSIA PJSC TOTAL,NaN,SBER,ALOR,RUB,1,NaN,203004.89


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Aug2024,Market Value
60,NOVATEK PJSC TOTAL,NaN,NVTK,ALOR,RUB,1,NaN,147215.87


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Aug2024,Market Value
75,ROSNEFT OIL CO PJSC TOTAL,NaN,ROSN,ALOR,RUB,1,NaN,101208.36


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Aug2024,Market Value
88,TATNEFT PJSC TOTAL,NaN,TATN,ALOR,RUB,1,NaN,42664.78


In [31]:
symbols_with_curr_vals[jpm_sp_col] = symbols_with_curr_vals[value_col] / symbols_with_curr_vals[holding_col]
symbols_with_curr_vals

,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Aug2024,Market Value,SP JPM GBP
0,ABSA GROUP LTD,BFX05H3,ABG,JSE,ZAC,1,35367,266436.50,7.533478
1,ABU DHABI COMMERCIAL BANK PJSC COMMON STOCK AED 1,6545464,ADCB,ADX,AED,1,148768,268435.89,1.804393
2,ABU DHABI ISLAMIC BANK PJSC COMMON STOCK AED 1,6001728,ADIB,ADX,AED,1,98209,255537.03,2.601972
3,ADES HOLDING CO COMMON STOCK SAR 1,BR56KM3,2382,TADAWUL,SAR,1,39150,162256.61,4.144486
4,ADNOC DRILLING CO PJSC COMMON STOCK AED,BN12D39,ADNOCDRILL,ADX,AED,1,250715,226453.85,0.903232
...,...,...,...,...,...,...,...,...,...
97,UNITED INTERNATIONAL TRANSPORTATION CO COMMON ...,B2493D0,4260,TADAWUL,SAR,1,9725,179243.42,18.4312
98,VTB BANK PJSC COMMON STOCK RUB 0.01,0H5837S,VTBR,ALOR,RUB,1,4246654,39097.97,0.009207
99,X 5 RETAIL GROUP NV-REGS GDR,0H6365S,FIVE,ALOR,RUB,1,275182,1112.67,0.004043
100,YANDEX NV COMMON STOCK USD 0.01,0H6379S,YDEX,ALOR,RUB,1,169819,24473.06,0.144113


In [33]:
symbols_with_curr_vals.to_csv(path_or_buf=f"../data/jema_symbols_with_vals_{year}{month_num}.csv")

In [34]:
import time

from tvDatafeed import TvDatafeed, Interval

tv = TvDatafeed()

for index, row in symbols_with_curr_vals.iterrows():

    # time.sleep(10)

    symbol = row.Symbol
    exchange = row.Exchange

    hist = tv.get_hist(
        symbol=symbol,
        exchange=exchange,
        interval=Interval.in_daily,
        n_bars=100
    )

    try:
        last_sp = hist.resample('ME').last().loc[f'{year}-{month_num}'].close.item()
        sp_now = hist.iloc[-1].close.item()
        
        symbols_with_curr_vals.loc[index, last_sp_col] = last_sp
        symbols_with_curr_vals.loc[index, today_sp_col] = sp_now
    except:
        print(f"Failed: {symbol} - {exchange}")

symbols_with_curr_vals

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Failed: 1150 - TADAWUL


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Failed: 0RCS - LSIN


ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


...

Failed: NVTK - ALOR
Failed: OTP - PSECZ


ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Failed: 0DP0 - LSE


/home/craigc/personal_workspace/FinanceDump/finance-venv/lib/python3.12/site-packages/tvDatafeed/main.py:136: SyntaxWarning: invalid escape sequence '\['
  out = re.search('"s":\[(.+?)\}\]', raw_data).group(1)
/home/craigc/personal_workspace/FinanceDump/finance-venv/lib/python3.12/site-packages/tvDatafeed/main.py:142: SyntaxWarning: invalid escape sequence '\['
  xi = re.split("\[|:|,|\]", xi)


KeyboardInterrupt: 

In [ ]:
symbol= 'KOMB'
exchange = 'PSECZ'

hist = tv.get_hist(
    symbol=symbol,
    exchange=exchange,
    interval=Interval.in_daily,
    n_bars=100
)

display(hist)

jun3124_sp = hist.resample('ME').last().loc[f'{year}-{month_num}' ].close.item()
sp_now = hist.iloc[-1].close.item()

print(jun3124_sp)
print(sp_now)


In [9]:
symbols_with_curr_vals.to_csv(path_or_buf="jema_symbols_with_vals_jun24.csv")

In [ ]:
currencies = list(symbols_with_curr_vals[currency_col].unique())

currencies_jun24 = {}
currencies_today = {}

for currency in currencies:
    if currency in currencies_jun24:
        continue
    try:
        data = tv.get_hist(symbol=f'{currency}GBP', exchange='FX_IDC', interval=Interval.in_daily, n_bars=100)
        currencies_jun24[currency] = data.loc['2024-06-30'].close.item()
        currencies_today[currency] = data.iloc[-1].close.item() 
    except:
        print(f"Failed: {currency}GBP")

display(currencies_jun24)
display(currencies_today)

In [ ]:
for index, row in symbols_with_curr_vals.iterrows():
    currency = row.Currency
    symbols_with_curr_vals.loc[index, 'fx gbp jun 24'] = currencies_jun24[currency]
    symbols_with_curr_vals.loc[index, 'fx gbp today'] = currencies_today[currency]

symbols_with_curr_vals

In [31]:
symbols_with_curr_vals.to_csv(path_or_buf="jema_symbols_with_vals_jun24.csv")

In [ ]:
for index, row in symbols_with_curr_vals.iterrows():
    fx_rate = row['fx gbp jun 24']
    if row.Currency == 'ZAR':
        fx_rate = fx_rate / 100
    sp_jpm = row[jpm_sp_col]
    sp_jun24 = row[jun24_sp_col]
    sp_jun24_gbp = sp_jun24 * fx_rate
    error = abs(sp_jpm - sp_jun24_gbp) / sp_jpm
    symbols_with_curr_vals.loc[index, 'SP Jun 24 GBP'] = sp_jun24_gbp
    symbols_with_curr_vals.loc[index, 'Error'] = error

symbols_with_curr_vals